In [2]:
# NOTA: Este bloque es para correr en Google Colab.
# Si se ejecuta localmente, ajustar base_path a la carpeta donde estén los CSV.
# Montamos Google Drive para poder acceder a los archivos almacenados en mi carpeta de Drive desde Google Colab.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Importamos pandas para manejar los dataframes
import pandas as pd

# Definimos la ruta base donde guarde los archivos CSV del INEGI ENOE
base_path = "/content/drive/MyDrive/Examen Final Ciencia de Datos/enoe_2025_trim2_csv/"

# Cargamos cada uno de los archivos de microdatos ENOE
# Usamos encoding='latin1' porque los archivos del INEGI suelen estar en ese
df_coei = pd.read_csv(base_path + "ENOE_COE1T225.csv", encoding='latin1')
df_coet = pd.read_csv(base_path + "ENOE_COE2T225.csv", encoding='latin1')
df_sdemt = pd.read_csv(base_path + "ENOE_SDEMT225.csv", encoding='latin1')
df_hogt = pd.read_csv(base_path + "ENOE_HOGT225.csv", encoding='latin1')
df_vivt = pd.read_csv(base_path + "ENOE_VIVT225.csv", encoding='latin1')

# Visualizamos las primeras filas para confirmar carga correcta
df_coei.head()

/tmp/ipython-input-1196630803.py:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hogt = pd.read_csv(base_path + "ENOE_HOGT225.csv", encoding='latin1')


,r_def,cd_a,ent,con,upm,d_sem,n_pro_viv,v_sel,n_hog,h_mud,...,p5f12,p5f13,p5f14,p5f15,p5f99,ur,tipo,mes_cal,fac_tri,fac_men
0,0,1,9,40004,906383,311,64,53,1,0,...,,,14,,,1,1,6,721,2311
1,0,1,9,40009,908908,306,123,80,1,0,...,,,14,,,1,2,5,426,2719
2,0,1,9,40009,908908,306,123,80,1,0,...,,,14,,,1,2,5,426,2719
3,0,1,9,40010,909336,313,102,53,1,0,...,,,,,,1,1,6,633,3555
4,0,1,9,40010,909336,313,102,53,1,0,...,,,,,,1,1,6,633,3555


In [4]:
# Revisamos cuántos registros y columnas tiene cada archivo
df_coei.shape
df_coet.shape
df_sdemt.shape
df_hogt.shape
df_vivt.shape

(150442, 25)

In [5]:
# Ver qué columnas comparten COEI y SDEMT. Esto sirve para saber con qué llaves podemos unir ambas tablas (personas).
df_coei.columns
df_sdemt.columns

Index(['r_def', 'loc', 'mun', 'est', 'est_d_tri', 'est_d_men', 'ageb',
       't_loc_tri', 't_loc_men', 'cd_a',
       ...
       'scian', 't_tra', 'emp_ppal', 'tue_ppal', 'trans_ppal', 'mh_fil2',
       'mh_col', 'sec_ins', 'tipo', 'mes_cal'],
      dtype='object', length=114)

In [6]:
# ============================================
# Exploración inicial para verificar columnas
# (Únicamente se utilizó para identificar llaves comunes entre COEI y SDEMT)
# No forma parte del pipeline final del modelo.
# ============================================

df_coei.columns[:30], df_sdemt.columns[:30]

(Index(['r_def', 'cd_a', 'ent', 'con', 'upm', 'd_sem', 'n_pro_viv', 'v_sel',
        'n_hog', 'h_mud', 'n_ent', 'per', 'n_ren', 'eda', 'n_inf', 'p1', 'p1a1',
        'p1a2', 'p1a3', 'p1b', 'p1c', 'p1d', 'p1e', 'p2_1', 'p2_2', 'p2_3',
        'p2_4', 'p2_9', 'p2a_dia', 'p2a_sem'],
       dtype='object'),
 Index(['r_def', 'loc', 'mun', 'est', 'est_d_tri', 'est_d_men', 'ageb',
        't_loc_tri', 't_loc_men', 'cd_a', 'ent', 'con', 'upm', 'd_sem',
        'n_pro_viv', 'v_sel', 'n_hog', 'h_mud', 'n_ent', 'per', 'n_ren',
        'c_res', 'par_c', 'sex', 'eda', 'nac_dia', 'nac_mes', 'nac_anio',
        'l_nac_c', 'cs_p12'],
       dtype='object'))

In [7]:
# Elegimos las llaves de identificación de persona que existen en ambos dataframes.

possible_keys = [
    'r_def', 'cd_a', 'ent', 'con', 'upm', 'd_sem',
    'n_pro_viv', 'v_sel', 'n_hog', 'h_mud',
    'n_ent', 'per'
]

# Nos quedamos solo con las que de verdad están presentes en ambas tablas
keys = [c for c in possible_keys if c in df_coei.columns and c in df_sdemt.columns]
print("Llaves que sí existen en ambos dataframes:", keys)


Llaves que sí existen en ambos dataframes: ['r_def', 'cd_a', 'ent', 'con', 'upm', 'd_sem', 'n_pro_viv', 'v_sel', 'n_hog', 'h_mud', 'n_ent', 'per']


In [8]:
# Unimos COEI (info laboral) con SDEMT (info sociodemográfica)
# Cada fila del nuevo df será una persona con ambas informaciones.

df = df_coei.merge(
    df_sdemt,
    on=keys,
    how='inner',               # inner: nos quedamos solo con personas que aparecen en ambos
    suffixes=('_coei', '_sdem')
)

# Revisamos tamaño del dataframe final
df.shape


(1409384, 274)

In [9]:
# ============================================
# Exploración inicial de los microdatos del INEGI
# (Se revisaron columnas y tamaño para confirmar integridad del archivo)
# Esta sección no forma parte del modelo final.
# ============================================

df.shape
df.columns[:40]

Index(['r_def', 'cd_a', 'ent', 'con', 'upm', 'd_sem', 'n_pro_viv', 'v_sel',
       'n_hog', 'h_mud', 'n_ent', 'per', 'n_ren_coei', 'eda_coei', 'n_inf',
       'p1', 'p1a1', 'p1a2', 'p1a3', 'p1b', 'p1c', 'p1d', 'p1e', 'p2_1',
       'p2_2', 'p2_3', 'p2_4', 'p2_9', 'p2a_dia', 'p2a_sem', 'p2a_mes',
       'p2a_anio', 'p2b_dia', 'p2b_sem', 'p2b_mes', 'p2b_anio', 'p2b', 'p2c',
       'p2d1', 'p2d2'],
      dtype='object')

In [10]:
# ============================================
# Selección de columnas para el modelo
# ============================================

# Variable objetivo: p1 (1 = ocupado, 0 = no ocupado)
df["empleo"] = df["p1"].apply(lambda x: 1 if x == 1 else 0)

# --- Display all column names to identify correct ones ---
print("Available columns in df:")
print(df.columns)
# --------------------------------------------------------

# Features que se usaran
cols_modelo = [
    "sex",          # sexo
    "eda_sdem",     # edad
    "anios_esc",    # escolaridad
    "ur_coei",      # urbano/rural
    "n_hog",        # tamaño del hogar
    "n_pro_viv",    # número de viviendas
    "h_mud",        # si se mudó recientemente
]

# Filtramos solo las columnas necesarias
df_model = df[cols_modelo + ["empleo"]].copy()

# Vemos cómo quedó
df_model.head(), df_model.shape

Available columns in df:
Index(['r_def', 'cd_a', 'ent', 'con', 'upm', 'd_sem', 'n_pro_viv', 'v_sel',
       'n_hog', 'h_mud',
       ...
       't_tra', 'emp_ppal', 'tue_ppal', 'trans_ppal', 'mh_fil2', 'mh_col',
       'sec_ins', 'tipo_sdem', 'mes_cal_sdem', 'empleo'],
      dtype='object', length=275)


(  sex eda_sdem  anios_esc  ur_coei  n_hog  n_pro_viv  h_mud  empleo
 0   2       57         16        1      1         64      0       1
 1   2       32         12        1      1        123      0       0
 2   1       43          9        1      1        123      0       0
 3   2        9          0        1      1        123      0       0
 4   2       32         12        1      1        123      0       1,
 (1409384, 8))

In [11]:
# =========================================
# Preparación de datos para el modelo
# =========================================

# Para evitar que Colab se trabe(porque ya se me trabo como 5 veces jejeje), usamos una muestra del dataframe:
df_small = df_model.sample(20000, random_state=42)  # 20,000 filas son más que suficientes

# Separamos features (X) y variable objetivo (y)
X = df_small.drop(columns=["empleo"])
y = df_small["empleo"]

# =========================================
# Limpieza de datos (convertimos todo a numérico)
# =========================================

for col in X.columns:
    # Convertimos a número; si hay errores los convierte en NaN
    X[col] = pd.to_numeric(X[col], errors='coerce')

    # Rellenamos los NaN para evitar que el modelo falle
    X[col] = X[col].fillna(0)

# =========================================
# División de datos
# =========================================

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =========================================
# Entrenamos modelo Random Forest
# =========================================

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    random_state=42
)

model.fit(X_train, y_train)

# =========================================
# Evaluación del modelo
# =========================================

from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy del modelo:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))


Accuracy del modelo: 0.55325

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.55      0.38      0.45      1920
           1       0.55      0.72      0.63      2080

    accuracy                           0.55      4000
   macro avg       0.55      0.55      0.54      4000
weighted avg       0.55      0.55      0.54      4000



In [12]:
# =========================================
# Importancia de variables del modelo
# =========================================

import pandas as pd
import numpy as np

importances = model.feature_importances_
feature_names = X.columns

df_importances = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

df_importances.head(20)

,feature,importance
5,n_pro_viv,0.391824
1,eda_sdem,0.364419
2,anios_esc,0.158882
3,ur_coei,0.027850
0,sex,0.027798
6,h_mud,0.021106
4,n_hog,0.008122


In [13]:
# Guardamos SOLO las columnas predictoras (sin empleo)
columnas_modelo = [c for c in df_model.columns if c != "empleo"]

import joblib
joblib.dump(model, "modelo_empleo.pkl")
joblib.dump(columnas_modelo, "columnas.pkl")

['columnas.pkl']

In [14]:
# Tomamos una muestra de 20,000 registros del df_model ya limpio
df_muestra = df_model.sample(20000, random_state=42)

# Guardamos la muestra en un CSV para usarlo en Streamlit
df_muestra.to_csv("df_model_muestra.csv", index=False)

In [15]:
# ============================================
# Cargamos la muestra del dataset preparado
# para alimentar el modelo de reclutamiento
# --------------------------------------------
# df_model_muestra.csv contiene 20,000 registros
# con las variables estructurales seleccionadas:
# - sex (sexo)
# - eda_sdem (edad)
# - anios_esc (años de escolaridad aprobados)
# - ur_coei (zona: urbano / rural)
# - n_hog (tamaño del hogar)
# - n_pro_viv (número de viviendas en el predio)
# - h_mud (movilidad reciente)
# - empleo (variable objetivo)
# ============================================

import pandas as pd
df = pd.read_csv("df_model_muestra.csv")
df.head()

,sex,eda_sdem,anios_esc,ur_coei,n_hog,n_pro_viv,h_mud,empleo
0,2,67,9,1,1,67,0,1
1,1,43,6,1,1,45,0,1
2,2,34,6,1,1,1,0,1
3,2,18,9,2,1,122,1,0
4,2,45,6,1,1,55,0,1
